In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("SvnLocalSpark") \
    .config("spark.jars.packages","io.delta:delta-spark_2.13:3.3.1,io.unitycatalog:unitycatalog-spark_2.13:0.2.1")\
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension,io.delta.sql.DeltaSparkSessionExtension")\
    .config("spark.sql.catalog.spark_catalog", "io.unitycatalog.spark.UCSingleCatalog")\
    .config("spark.sql.catalog.unity","io.unitycatalog.spark.UCSingleCatalog")\
    .config("spark.sql.catalog.unity.uri","http://localhost:8080/api/2.1/unity-catalog/")\
    .config("spark.sql.catalog.unity.token","")\
    .config("spark.sql.catalog.unity.type","rest")\
    .config("spark.sql.catalog.unity.warehouse", "unity")\
    .config("spark.sql.warehouse.dir", "file:///C:/Users/SjoerdvanNoort/ReposSVN/Data/UC/")\
    .config("spark.databricks.delta.catalog.update.enabled", "true")\
    .config("spark.databricks.csv.catalog.update.enabled", "true")\
    .config("spark.sql.defaultCatalog","unity")\
    .master("local")\
    .getOrCreate()

print(f"spark {spark.version} {spark.sparkContext.uiWebUrl}")

spark 3.5.4 http://DESKTOP-4GOMK6M:4040


In [2]:
%load_ext sparksql_magic

In [3]:
%%sparksql
SHOW SCHEMAS

namespace
default
landing
vanilladwh


In [ ]:
%%sparksql
SHOW TABLES IN default

In [4]:
%%sparksql
SELECT * FROM default.user_countries

only showing top 20 row(s)


first_name,age,country
Crystal Bruce,24,Serbia
Anthony Gonzales,71,Serbia
Christina Hall,69,Serbia
Colleen Spencer,34,Serbia
Brian Smith,24,Serbia
Scott Mcpherson III,74,Serbia
Michael Wolfe,30,Serbia
Virginia Burton,38,Serbia
Rodney Pope,74,Serbia
Gary Wright,74,Serbia


In [ ]:
%%sparksql
SELECT country , count(1) nr_users, avg(age) avg_age
FROM default.user_countries
GROUP BY country

In [ ]:
%%sparksql
CREATE SCHEMA IF NOT EXISTS landing

In [ ]:
spark.catalog.getDatabase("landing")
spark.catalog.createTable(
    tableName = "landing.commercial_properties",
    source = "csv",
    description = "property values",
    header="true", delimiter=",", path="/Users/SjoerdvanNoort/ReposSVN/svn-local-spark/resources/sourcedata/commercial_property_snapshots_100_M39.csv", inferSchema="true")

In [ ]:
spark.catalog.getDatabase("landing")
spark.catalog.createTable(
    tableName = "landing.commercial_propertiesV2",
    source = "csv",
    description = "property values",
    header="true", delimiter=",", location="file:///C:/Users/SjoerdvanNoort/ReposSVN/svn-local-spark/resources/sourcedata/commercial_property_snapshots_100_M39.csv", inferSchema="true")

In [ ]:
spark.catalog.getDatabase("landing")
spark.catalog.createTable(
    tableName = "landing.commercial_propertiesV3",
    source = "csv",
    description = "property values",
    header="true", delimiter=",", path="file:///C:/Users/SjoerdvanNoort/ReposSVN/svn-local-spark/resources/sourcedata/commercial_property_snapshots_100_M39.csv", inferSchema="true")

In [ ]:
%%sparksql
CREATE TABLE landing.test1(
    ColA string,
    ColB string,
    ColC double
)
USING delta
LOCATION 'file:///C:/Users/SjoerdvanNoort/ReposSVN/Data/UC/landing/test1'

In [ ]:
%%sparksql
USE landing

In [ ]:
%%sparksql
SELECT * 
FROM unity.landing.commercial_propertiesV3

In [ ]:
%%sparksql
CREATE SCHEMA IF NOT EXISTS vanilladwh
LOCATION 'file:///C:/Users/SjoerdvanNoort/ReposSVN/Data/UC/vanilla_data'

In [ ]:
from datetime import datetime, date
from pyspark.sql.functions import lit

current_date = datetime.strptime("2023-01-01", '%Y-%m-%d')
file_path = f"../resources/generated/commercial_property/{current_date.strftime('%Y')}/{current_date.strftime('%m')}/commercial_property_snapshot_{current_date.strftime('%Y%m%d')}.csv"
inp = spark.read.option("header", True).option("inferSchema", True).csv(file_path)\
    .withColumn("date", lit(current_date))

#inp.show()S

#inp.write.format("delta").option('path','file:///C:/Users/SjoerdvanNoort/ReposSVN/Data/UC/vanilla_data/test2').saveAsTable("vanilladwh.test2")
inp.write.format("delta").saveAsTable("unity.vanilladwh.test2")

In [ ]:
%%sparksql
SELECT * from vanilladwh.test2